# Installing the environment

In [1]:
from google.colab import drive

drive.mount('/content/drive')
f = open("drive/MyDrive/api_key.txt", "r")
API_KEY=f.readline()
f.close()

Mounted at /content/drive


In [2]:
try:
  import openai
except:
  !pip install openai==1.42.0
import openai

import os
os.environ['OPENAI_API_KEY'] =API_KEY
openai.api_key = os.getenv("OPENAI_API_KEY")

In [3]:
!pip install jsonlines==4.0.0

In [4]:
!pip install datasets==2.20.0

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x

# Preparing the dataset for fine-tuning

In [ ]:
# Import required libraries
from datasets import load_dataset
import pandas as pd

# Load the SciQ dataset from HuggingFace
dataset_view = load_dataset("sciq", split="train")

# Filter the dataset to include only questions with support and correct answer
filtered_dataset = dataset_view.filter(lambda x: x["support"] != "" and x["correct_answer"] != "")


# Print the number of questions with support
print("Number of questions with support: ", len(filtered_dataset))

Number of questions with support:  10481


In [ ]:
# Convert the filtered dataset to a pandas DataFrame
df_view = pd.DataFrame(filtered_dataset)

# Columns to drop
columns_to_drop = ['distractor3', 'distractor1', 'distractor2']

# Dropping the columns from the DataFrame
df_view = df_view.drop(columns=columns_to_drop)

# Display the DataFrame
df_view.head()

,question,correct_answer,support
0,What type of organism is commonly used in prep...,mesophilic organisms,"Mesophiles grow best in moderate temperature, ..."
1,What phenomenon makes global winds blow northe...,coriolis effect,Without Coriolis Effect the global winds would...
2,Changes from a less-ordered state to a more-or...,exothermic,Summary Changes of state are examples of phase...
3,What is the least dangerous radioactive decay?,alpha decay,All radioactive decay is dangerous to living t...
4,Kilauea in hawaii is the world’s most continuo...,smoke and ash,Example 3.5 Calculating Projectile Motion: Hot...


#  Preparing the dataset for fine-tuning

In [ ]:
import json
import jsonlines
import pandas as pd
from datasets import load_dataset

# Load and clean the dataset as previously described
dataset = load_dataset("sciq", split="train")
filtered_dataset = dataset.filter(lambda x: x["support"] != "" and x["correct_answer"] != "")

# Convert to DataFrame and clean
df = pd.DataFrame(filtered_dataset)
columns_to_drop = ['distractor3', 'distractor1', 'distractor2']
df = df.drop(columns=columns_to_drop)

# Prepare the data items for JSON lines file
items = []
for idx, row in df.iterrows():
    detailed_answer = row['correct_answer'] + " Explanation: " + row['support']
    items.append({
        "messages": [
            {"role": "system", "content": "Given a science question, provide the correct answer with a detailed explanation."},
            {"role": "user", "content": row['question']},
            {"role": "assistant", "content": detailed_answer}
        ]
    })

# Write to JSON lines file
with jsonlines.open('/content/QA_prompts_and_completions.json', 'w') as writer:
    writer.write_all(items)

Filter:   0%|          | 0/11679 [00:00<?, ? examples/s]

# Visualizing the JSON file

In [ ]:
dfile = "/content/QA_prompts_and_completions.json"

In [ ]:
import pandas as pd

df = pd.read_json(dfile, lines=True)
df

,messages
0,"[{'role': 'system', 'content': 'Given a scienc..."
1,"[{'role': 'system', 'content': 'Given a scienc..."
2,"[{'role': 'system', 'content': 'Given a scienc..."
3,"[{'role': 'system', 'content': 'Given a scienc..."
4,"[{'role': 'system', 'content': 'Given a scienc..."
...,...
10476,"[{'role': 'system', 'content': 'Given a scienc..."
10477,"[{'role': 'system', 'content': 'Given a scienc..."
10478,"[{'role': 'system', 'content': 'Given a scienc..."
10479,"[{'role': 'system', 'content': 'Given a scienc..."


# Fine-tuning the model

In [ ]:
from openai import OpenAI
import jsonlines
client = OpenAI()
# Uploading the training file

result_file = client.files.create(
  file=open("QA_prompts_and_completions.json", "rb"),
  purpose="fine-tune"
)

print(result_file)
param_training_file_name = result_file.id
print(param_training_file_name)

# Creating the fine-tuning job

ft_job = client.fine_tuning.jobs.create(
  training_file=param_training_file_name,
  model="gpt-4o-2024-08-06"
)

# Printing the fine-tuning job
print(ft_job)

FileObject(id='file-N6jSQ12gwVfDJWjKUmnPoD', bytes=8062970, created_at=1741227518, filename='QA_prompts_and_completions.json', object='file', purpose='fine-tune', status='processed', status_details=None, expires_at=None)
file-N6jSQ12gwVfDJWjKUmnPoD
FineTuningJob(id='ftjob-W4FXLl39U9TVStLFv6uAVZBZ', created_at=1741227518, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs='auto'), model='gpt-4o-2024-08-06', object='fine_tuning.job', organization_id='org-VoNEXm1Xg5gwuqkZcpfHHaLu', result_files=[], seed=1144370893, status='validating_files', trained_tokens=None, training_file='file-N6jSQ12gwVfDJWjKUmnPoD', validation_file=None, estimated_finish=None, integrations=[], method=Method(dpo=None, supervised=MethodSupervised(hyperparameters=MethodSupervisedHyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs='auto')), type='supervised'), use

# Monitoring the fine-tunes

In [11]:
import pandas as pd
from openai import OpenAI

client = OpenAI()
response = client.fine_tuning.jobs.list(limit=3)

# Initialize lists to store the extracted data
job_ids = []
created_ats = []
statuses = []
models = []
training_files = []
error_messages = []
fine_tuned_models = [] # List to store the fine-tuned model names

for job in response.data:
  job_ids.append(job.id)
  created_ats.append(job.created_at)
  statuses.append(job.status)
  models.append(job.model)
  training_files.append(job.training_file)
  error_message = job.error.message if job.error else None
  error_messages.append(error_message)

  # Append the fine-tuned model name
  fine_tuned_model = job.fine_tuned_model if hasattr(job, 'fine_tuned_model') else None
  fine_tuned_models.append(fine_tuned_model)

In [12]:
import pandas as pd

response = client.fine_tuning.jobs.list(limit=3)

df = pd.DataFrame({
  'Job ID': job_ids,
  'Created At': created_ats,
  'Status': statuses,
  'Model': models,
  'Training File': training_files,
  'Error Message': error_messages,
  'Fine-Tuned Model': fine_tuned_models # Include the fine-tuned model names
})

In [13]:
df['Created At'] = pd.to_datetime(df['Created At'], unit='s')
df = df.sort_values(by='Created At', ascending=False)

df

,Job ID,Created At,Status,Model,Training File,Error Message,Fine-Tuned Model
0,ftjob-dzCwKIX7fBInuyfFnMgSrC9K,2025-03-06 23:33:19,succeeded,gpt-4o-mini-2024-07-18,file-XVvGWsLHzbmm3BzKregQgp,None,ft:gpt-4o-mini-2024-07-18:personal::B8FkVoqc
1,ftjob-f4KovQv4BNdKCPivMm9GF22N,2025-03-06 02:44:32,failed,gpt-4o-mini-2024-07-18,file-6vLNXQQTw5MCdYDhurEEjb,The job failed due to an internal error.,None
2,ftjob-W4FXLl39U9TVStLFv6uAVZBZ,2025-03-06 02:18:38,cancelled,gpt-4o-2024-08-06,file-N6jSQ12gwVfDJWjKUmnPoD,None,None


In [14]:
import pandas as pd

generation=False  # until the current model is fine-tuned
# Attempt to find the first non-empty Fine-Tuned Model
non_empty_models = df[df['Fine-Tuned Model'].notna() & (df['Fine-Tuned Model'] != '')]

if not non_empty_models.empty:
    first_non_empty_model = non_empty_models['Fine-Tuned Model'].iloc[0]
    print("The latest fine-tuned model is:", first_non_empty_model)
    generation=True
else:
    first_non_empty_model='None'
    print("No fine-tuned models found.")

The latest fine-tuned model is: ft:gpt-4o-mini-2024-07-18:personal::B8FkVoqc


# Using the fine-tuned model

In [15]:
prompt = "What phenomenon makes global winds blow northeast to southwest or the reverse in the northern hemisphere and northwest to southeast or the reverse in the southern hemisphere?"


if generation:
  response = client.chat.completions.create(
      model=first_non_empty_model,
      temperature=0.0,
      messages=[
        {"role": "system", "content": "Given a question, reply with a complete explanation for students."},
        {"role": "user", "content": prompt}
      ]
  )
else:
  print("Error: model is none, cannot proceed with the API request.")

In [16]:
if generation:
  response_text = response.choices[0].message.content
  print(response_text)

coriolis effect Explanation: The Coriolis effect makes global winds blow northeast to southwest or the reverse in the northern hemisphere and northwest to southeast or the reverse in the southern hemisphere.


In [17]:
import textwrap

if generation:
  wrapped_text = textwrap.fill(response_text.strip(), 60)
  print(wrapped_text)

coriolis effect Explanation: The Coriolis effect makes
global winds blow northeast to southwest or the reverse in
the northern hemisphere and northwest to southeast or the
reverse in the southern hemisphere.
